### ERPA

hole-hole

$|\Psi_{\lambda}> = \sum_{pq} c_{pq,\lambda} a_q a_p$

GEVP to be solved

$\sum_{pq}c_{pq} <\Psi|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi> = \Delta E_{\lambda} \sum_{pq}c_{pq} <\Psi|[a^\dagger_r a^\dagger_s,a_q a_p]|\Psi>$

$A_{rs,pq} = <\Psi|[a^\dagger_r a^\dagger_s,[H,a_q a_p]]|\Psi>$

$M_{rs,pq} = <\Psi|[a^\dagger_r a^\dagger_s, a_q a_p]|\Psi>$

Overlap

$M_{rs,pq} = <\Psi|a^\dagger_r a^\dagger_s a_q a_p|\Psi>$

$C^T_{rs} M_{rs,pq} C_{pq} =? I$




In [1]:
import sys  
sys.path.insert(0, '../')

In [2]:
# Import modules
from src.scripts.script_erpa import run_eom, overlap
from src.scripts.script_erpa import brute_hherpa_lhs, wrap_pyci
# from src.scripts.tools_erpa import solve_dense, solve_lowdin
from eomee.solver import nonsymmetric, lowdin, lowdin_img
from eomee.tools import pickpositiveeig, spinize, symmetrize, antisymmetrize

import pyci

import numpy as np

Define test cases/systems

In [3]:
operator = 'hh'
test1 = ('Be_0_1/Be_0_1_3-21g', '3-21G')
test2 = ('be/be_sto-6g', 'STO-6G')
test3 = ('h2/h2_3-21g', '3-21G')
test4 = ('h2/h2_sto-6g', 'STO-6G')
test5 = ('HHHH_8_sp/HHHH_8_sp_sto-6g', 'STO-6G')
fname, basis = test1
nparts = (2,2)

dirname = f'../data/interim/integrals/{basis}'
h_ij = np.load(f"{dirname}/{fname}_oneint.npy")
g_ijkl = np.load(f"{dirname}/{fname}_twoint.npy")

In [4]:
### Get DOCI RDMs for ERPA
wfn_type = 'fci'
out = wrap_pyci(nparts, 0., h_ij, g_ijkl, wfn_type)
dm1, dm2 = (out['dm1'], out['dm2'])

In [5]:
# Solve the GEVP for hole-hole operator
gevp = run_eom(operator, nparts, h_ij, g_ijkl, one_dm=dm1, two_dm=dm2, restricted=True, tol=1.0e-7, orthog="nonsymm")
v = spinize(g_ijkl)
b_lhs = brute_hherpa_lhs(gevp['h'], v, gevp['dm1'], gevp['dm2'])

In [6]:
# w, cv = solve_dense(b_lhs, gevp['gevp'].rhs, tol=1.0e-7, orthog="asymmetric")
# ev, cv = solve_lowdin(b_lhs, gevp['gevp'].rhs, tol=1.0e-7) # OK
w, cv = nonsymmetric(b_lhs, gevp['gevp'].rhs,  tol=1.0e-7)
# w, cv = lowdin_img(b_lhs, gevp['gevp'].rhs,  tol=1.0e-7)

In [7]:
# Get the left- and right-hand side matrices for further manipulations bellow
n = gevp['gevp'].n
k = h_ij.shape[0]
lhs = b_lhs #gevp['gevp'].lhs
rhs = gevp['gevp'].rhs

Not really relevant to this notebook: Just checking that the RHS and LHS have the expected symmetries

Checking overlap matrix for hh-RPA/hh-ERPA

In [8]:
# States from positive side of spectrum
# pw, pcv, _ = pickpositiveeig(gevp['w'], gevp['cv'], tol=1.e-2)
pw, pcv, _ = pickpositiveeig(w, cv, tol=1.e-2)

# c^*_pq c_rs <|p^+ q^+ s r |>
metric = overlap(operator, gevp['dm1'], gevp['dm2'])
ovlp_ = np.dot(pcv, np.dot(metric, pcv.T))  # c(M,1).T * S(M,M) * c(M,1)

print('Diagonal elements of overlap matrix\n', np.diag(ovlp_)[:5])

Diagonal elements of overlap matrix
 [1.72325384 0.0018542  0.00183217 0.0018542  0.0018542 ]


In [9]:
print('Double IP\n', np.sort(pw)[:10])

Double IP
 [1.03633717 5.43314265 5.43314265 5.43314265 5.49711531 5.63819045
 5.63819045 5.63819045 5.63819045 5.63819045]


In [10]:
# Reconstruct 2-RDM from hh-TDMs and check trace

cv_mrs = pcv.reshape(pcv.shape[0], n, n)
tdms = np.einsum("mrs,pqsr->mpq", cv_mrs, gevp['gevp'].rhs.reshape(n,n,n,n)) #gevp['dm2']
tv = np.zeros_like(gevp['dm2']) #dm2
for rdm in tdms:
    tv += np.einsum("pq,rs->pqrs", rdm, rdm, optimize=True)
new_rdm2 =  tv/2

trdm2 = np.einsum("pjqj", new_rdm2)
print('2-RDm normalization condition\n', np.einsum("ijij", new_rdm2), (sum(nparts) * (sum(nparts) - 1)))

2-RDm normalization condition
 3.239116572659635 12


In [11]:
# print(np.diag(trdm2[:k,:k]))
# print()
# print(np.diag(trdm2[k:,k:]))

# assert np.allclose(trdm2[:k,:k], trdm2[k:,k:])
# assert np.allclose(new_rdm2[:k, :k, :k, :k], new_rdm2[k:, k:, k:, k:])

### Eliminate $a_q a_p |\Psi>$ configurations with $p>q$

Let

$M_{rs,pq}= [a^\dagger_r a^\dagger_s,[H, a_q a_p]]$ $\forall r < s, p < q$

In [12]:
# ### Get DOCI RDMs for ERPA
# pyci_dms = pyci_seniorityzero(nparts, 0., h_ij, g_ijkl)
# dm1 = pyci_dms['dm1']
# dm2 = pyci_dms['dm2']

# Get GEVP for hole-hole operator
# gevp2 = run_eom(operator, nparts, h_ij, g_ijkl, one_dm=gevp['dm1'], two_dm=gevp['dm2'], restricted=True, tol=1.0e-7, orthog="asymmetric")

In [13]:
# Redefining matrices
n_pq = n*(n-1)//2
A = np.zeros((n_pq, n_pq))
M = np.zeros((n_pq, n_pq))

configs = []
for i in range(n):
    for j in range(i+1,n):
        ij = n*i + j
        configs.append(ij)

for i, rs in enumerate(configs):
    for j, pq in enumerate(configs):
        A[i,j] = b_lhs[rs,pq]   #gevp2['gevp'].lhs[rs,pq]
        M[i,j] = gevp['gevp'].rhs[rs,pq]


In [14]:
# assert np.allclose(A, A.T)
assert np.allclose(M, M.T)

In [15]:
# # Find indices of off diagonal elements
# arr = A
# indices = (A.shape[0], A.shape[0])
# off_diagonal = (arr - np.diag(np.diagonal(arr))).flatten()
# off_diagonal[np.abs(off_diagonal)<=1.0e-7] = 0.
# off_diagonal = off_diagonal.reshape(indices)
# n_off_diagonal = np.count_nonzero(off_diagonal)
# print('N off diagonal vals', n_off_diagonal)
# out_ind = np.transpose(np.nonzero(off_diagonal))
# # print(out_ind)
# for i,j in zip(out_ind, off_diagonal[np.nonzero(off_diagonal)]):
#     print(i, j)

In [16]:
# print(n_pq)
# print(len(configs))

# es, cs = solve_dense(A, M, tol=1.0e-7, orthog="asymmetric")
# es, cs = nonsymmetric(A, M,  tol=1.0e-7)
es, cs = lowdin_img(A, M,  tol=1.0e-7)

In [17]:
pes, pcs, _ = pickpositiveeig(es, cs, tol=1.e-2)
# print(np.sort(pes)[:10])

In [18]:
# Compare truncated GEVP spectrum to original one
print('Full GEVP    ', len(pw), '\n', np.sort(pw)[:5])
print('Truncated GEVP    ', len(pes), '\n', np.sort(pes)[:5])

Full GEVP     22 
 [1.03633717 5.43314265 5.43314265 5.43314265 5.49711531]
Truncated GEVP     22 
 [1.34804572 5.43324816 5.43324816 5.43326025 5.49838928]


In [19]:
# Check overlap: c^*_rs c_pq <|r^+ s^+ q p |>
I = np.eye(es.shape[0])

metric = np.zeros((n_pq, n_pq))
P_matrix = gevp['gevp'].rhs # gevp['dm2'].reshape(n**2,n**2) #
for i, rs in enumerate(configs):
    for j, pq in enumerate(configs):
        metric[i,j] = P_matrix[rs,pq]


# print(metric.shape)
# print(pcs.shape)
ovlp_ = np.dot(pcs, np.dot(metric, pcs.T))  # c(M,1).T * S(M,M) * c(M,1)


# print(ovlp_)
print('Diagonal elements of overlap matrix\n', np.diag(ovlp_)[:5])

# # Find off diagonal elements indices
# arr = ovlp_
# indices = (ovlp_.shape[0], ovlp_.shape[0])
# off_diagonal = (arr - np.diag(np.diagonal(arr))).flatten()
# off_diagonal[np.abs(off_diagonal)<=1.0e-7] = 0.
# off_diagonal = off_diagonal.reshape(indices)
# # n_off_diagonal = np.count_nonzero(off_diagonal)
# # print('N off diagonal vals', n_off_diagonal)
# out_ind = np.transpose(np.nonzero(off_diagonal))
# # print(out_ind)
# for i,j in zip(out_ind, off_diagonal[np.nonzero(off_diagonal)]):
#     print(i, j)

Diagonal elements of overlap matrix
 [0.87151049 0.99998371 0.99998371 0.89718377 0.99434839]


Considerations when reconstructed the 2-RDM from the truncated GEVP solutions:

$\Gamma_{pq,rs} = <\Psi|a^\dagger_p a^\dagger_q a_s a_r|\Psi>$

Elements of symmetry:

$\Gamma_{pq,rs} = \Gamma_{qp,sr} = \Gamma_{rs,pq}  = \Gamma_{sr,qp}$

$\Gamma_{pq,rs} = -\Gamma_{pq,sr} = -\Gamma_{qp,rs}$

symmetry for spin-components:

$\Gamma^{aaaa}_{pq,rs} = \Gamma^{aaaa}_{qp,sr} = \Gamma^{aaaa}_{rs,pq}  = \Gamma^{aaaa}_{sr,qp}$

$\Gamma^{aaaa}_{pq,rs} = -\Gamma^{aaaa}_{pq,sr} = -\Gamma^{aaaa}_{qp,rs}$

$\Gamma^{abab}_{pq,rs} = \Gamma^{abab}_{rs,pq}$

$\Gamma^{abab}_{pq,rs} = \Gamma^{baba}_{qp,sr}$

ERPA $\Gamma$:

$\Gamma_{pq,rs} = <\Psi|a^\dagger_p a^\dagger_q a_s a_r|\Psi>\\ = \sum_n <\Psi|a^\dagger_p a^\dagger_q |\Psi^{N-2}_n><\Psi^{N-2}_n| a_s a_r|\Psi>\\ = \sum_n \gamma_{pq;n} \gamma_{rs;n}$

where $\gamma_{pq;n}$ is a transition density matrix element:

$\gamma_{pq;n} = <\Psi|a^\dagger_p a^\dagger_q |\Psi^{N-2}_n>\\ \approx \sum_{ij} c_{ij;n} <\Psi|a^\dagger_p a^\dagger_q a_j a_i|\Psi>\\
= \sum_{ij} c_{ij;n} M_{pq,ij}$

Since only the configuration with $p<q$ are considered I'm directly evaluating the following elements of Gamma:

$\Gamma_{p<q,r<s} = \sum_n \gamma_{p<q;n} \gamma_{r<s;n}$

but need to fill the remaining term considering the symmetry elements above.

When truncating the configuration space I used the following convention:

Linearize pair of indexes (p, q) as:

$pq = 2k \times p + q$ $\forall p < q$

by redefining the compound indexes as for example $pq = X$, I can write:

$\Gamma_{pq,rs} = \Gamma_{X,Y} \approx \sum_n \gamma_{X;n} \gamma_{Y;n}$ $\forall p < q, r<s$

so that

$\gamma_{pq;n} = <\Psi|a^\dagger_p a^\dagger_q |\Psi^{N-2}_n>\\ \approx \sum_{ij} c_{ij;n} <\Psi|a^\dagger_p a^\dagger_q a_j a_i|\Psi>\\
= \sum_{ij} c_{ij;n} M_{pq,ij}\\ = \sum_Y C_{Y;n} M_{XY} = \gamma_{X;n}$

In [20]:
# Verifica: elementos aaaa, bbbb correspondientes de la matriz metrica truncada; 
# elementos aa, bb correspondientes de la matriz de coefficientes c_iaja ? c_ibjb
# elementos aa, bb correspondientes de la matriz de transicion para unestado (o todos)

# Reconstruct 2RDM
pes, pcs, _ = pickpositiveeig(es, cs, tol=1.e-2)

#
# gamma_x,n = \sum_y c_y,n M_xy = tdms_n
#
tdms = np.einsum("ny,xy->nx", pcs, M)
temp = np.zeros_like(gevp['dm2'])#.reshape(n**2, n**2)


In [21]:
# Fill terms Gamma_pq,rs --> Gamma_x,y
# Gamma_x,y is symmetric with Gamma_pq,rs = Gamma_rs,pq
temp = np.zeros((n_pq, n_pq))
for rdm in tdms:
    temp += np.outer(rdm, rdm)
# Unfold into 4-index tensor Gamma_p,q,r,s
# Fill missing symmetric terms p>q, r>s
dm2rpa = np.zeros_like(gevp['dm2'])
for x, pq in enumerate(configs):
    for y, rs in enumerate(configs):
        (p,q) = (pq // n, pq % n)
        (r,s) = (rs // n, rs % n)
        dm2rpa[p,q,r,s] = temp[x,y]
        dm2rpa[q,p,s,r] = temp[x,y]

dm2rpa = antisymmetrize(dm2rpa)

tr_dm2 = np.einsum("pqps->qs", dm2rpa)
print('2-RDm normalization condition\n', np.einsum("ijij", dm2rpa), (sum(nparts) * (sum(nparts) - 1)))


# tv = np.zeros_like(gevp['dm2']).reshape(n**2, n**2)
# for rdm in tdms:
#     for i, rs in enumerate(configs):
#         for j, pq in enumerate(configs):
#             tv[rs,pq] += rdm[i] * rdm[j]
# #             # (r,s) = (rs//n, rs % n)
# #             # (p,q) = (pq//n, pq % n)
# #             # if ((r <= k-1) and (s <= k-1)) and ((p <= k-1) and (q <= k-1)):
# #             #     print(rdm[i], rdm[j], tv[rs,pq])
# #             # # if ((r <= k-1) and (s > k-1)) and ((p <= k-1) and (q > k-1)):
# #             # #     print(rdm[i], rdm[j], tv[rs,pq])
# #             # # if ((r > k-1) and (s > k-1)) and ((p > k-1) and (q > k-1)):
# #             # #     print(rdm[i], rdm[j], tv[rs,pq])
# #             # # if ((r > k-1) and (s <= k-1)) and ((p > k-1) and (q <= k-1)):

# # OJO missing elementos de simetria
# new_dm2 = 2*tv.reshape(n,n,n,n)
# # # t_rdm2 = np.einsum("pqps", new_dm2)
# # t_rdm2 = np.einsum("pqrq->pr", new_dm2)
# print('2-RDm normalization condition\n', np.einsum("ijij", new_dm2), (sum(nparts) * (sum(nparts) - 1)))

2-RDm normalization condition
 11.118969636249007 12


In [22]:
print(np.diag(tr_dm2[:k,:k]))
print()
print(np.diag(tr_dm2[k:,k:]))
assert np.allclose(dm2rpa[:k, :k, :k, :k], dm2rpa[k:, k:, k:, k:])

[2.87249482e+00 2.51037041e+00 5.66770359e-02 5.88788936e-02
 4.08485557e-02 6.34635897e-03 5.86877321e-03 5.78823732e-03
 2.85398925e-03]

[2.87174384e+00 2.51052383e+00 5.74109188e-02 3.70895039e-02
 6.12822323e-02 5.07815859e-03 6.96100376e-03 5.90168936e-03
 2.85136882e-03]


AssertionError: 

In [ ]:
# for i, rs in enumerate(configs):
#     for j, pq in enumerate(configs):
#         (r,s) = (rs//n, rs % n)
#         (p,q) = (pq//n, pq % n)
#         # if ((r <= k-1) and (s <= k-1)) and ((p <= k-1) and (q <= k-1)):
#         # if ((r <= k-1) and (s > k-1)) and ((p <= k-1) and (q > k-1)):
#         # if ((r > k-1) and (s > k-1)) and ((p > k-1) and (q > k-1)):
#         if ((r > k-1) and (s <= k-1)) and ((p > k-1) and (q <= k-1)):
#             print(r,s,p,q, M[i,j], gevp2['gevp'].rhs[rs,pq])

Solve hh-ERPA eliminatig the elements with index ($p \bar{p}, q \bar{q}$) from the left and RHS of the GEVP

In [ ]:
# # Get GEVP for hole-hole operator
# gevp3 = run_eom(nparts, h_ij, g_ijkl, one_dm=None, two_dm=None, operator=op, restricted=True, tol=1.0e-7, orthog="asymmetric")


# # Redefining matrices
# n_pq = n**2 - n
# A = np.zeros((n_pq, n_pq))
# M = np.zeros((n_pq, n_pq))

# configs = []
# for p in range(n):
#     for q in range(p+1,n):
#         if q == p + k:
#             continue
#         pq = n*p + q
#         configs.append(pq)

# for i, rs in enumerate(configs):
#     for j, pq in enumerate(configs):
#         A[i,j] = gevp3['gevp'].lhs[rs,pq]
#         M[i,j] = gevp3['gevp'].rhs[rs,pq]

# # Evaluate new 2-RDM and check trace
# ev, cv = solve_dense(A, M, tol=1.0e-7, orthog="asymmetric")

# pw, pc, _ = pickpositiveeig(ev, cv, tol=1.e-2)
# print('Double IP\n', np.sort(pw))

In [ ]:
# # Check overlap: c^*_rs c_pq <|r^+ s^+ q p |>
# I = np.eye(n_pq)

# metric = np.zeros((n_pq, n_pq))
# P_matrix = gevp3['dm2'].reshape(n**2,n**2)
# for i, rs in enumerate(configs):
#     for j, pq in enumerate(configs):
#         metric[i,j] = P_matrix[rs,pq]

# # ovlp = np.dot(cv, np.dot(metric, cv.T))  # c(M,1).T * S(M,M) * c(M,1)
# ovlp_ = np.dot(pc, np.dot(metric, pc.T))

# # print(ovlp_)
# print('Diagonal elements of overlap matrix\n', np.diag(ovlp_))

In [ ]:
# tdms = np.einsum("mj,ij->mi", pc, M)
# tv = np.zeros_like(gevp3['dm2']).reshape(n**2, n**2)

# for rdm in tdms:
#     for i, rs in enumerate(configs):
#         for j, pq in enumerate(configs):
#             tv[rs,pq] += rdm[i] * rdm[j]

     
# # OJO missing elementos de simetria
# new_dm2 = tv.reshape(n,n,n,n)
# tr_dm2 = np.einsum("pqrq->pr", new_dm2)

# print('2-RDm normalization condition\n', np.einsum("ijij", new_dm2), (sum(nparts) * (sum(nparts) - 1)))

In [ ]:
# print(np.diag(tr_dm2[:k,:k]))
# print()
# print(np.diag(tr_dm2[k:,k:]))
# assert np.allclose(new_dm2[:k, :k, :k, :k], new_dm2[k:, k:, k:, k:])

In [ ]:
# assert np.allclose(new_dm2, new_dm2.transpose((1,0,3,2))) # ijkl, jilk
assert np.allclose(new_dm2, new_dm2.transpose((2,3,0,1))) # ijkl, klij
# assert np.allclose(new_dm2, new_dm2.transpose((3,2,1,0))) # ijkl, jilk
# assert np.allclose(new_dm2, -new_dm2.transpose((0,1,3,2))) # ijkl, ijlk
# assert np.allclose(new_dm2, -new_dm2.transpose((1,0,2,3))) # ijkl, ijlk


<!-- DONE: Que problema hay con la simetria del GEVP que no da como se espera. Como ultimo recurso revisar la ecuacion hh implementada :(
    Luego de implementar literalmente las ecuaciones del LHS de hh-ERPA, se recupera la simetria esperada de la matriz (salvo con la funcion DOCI enla que la permutacion klji != jikl por alguna razon). Sin embargo esto no cambia el resultado de los overlaps o de la normalizacion incorrecta de la 2RDM reconstruida.


Resultados hasta ahora con el procedimiento de resolver hh-GEVP solo con estados p>q (que denominare GEVP truncado):
Para HF referencia: obtengo el mismo numero de mismos autovalores positivos que en el GEVP original con un factor de 1/2 de diferencia.
De manera semejante los autovectores son correspondientes y nuevamente tienen un factor de 1/2
La traza de la 2rdm reconstruida pierde la submatriz con componente betabeta, en tanto que la componente alphaalpha es exactamente 1/2 de la que se obtiene con el procedimiento para el GEVP original. Consecuentemente la normalizacion de la 2rdm reconstruida es tiene un factor de 1/2.
Para DOCI: hay el mismo numero de soluciones positivas que en GEVP, pero los valores no son iguales, en el caso de Be los 5 autovalores de menor energia se asemejan a las soluciones de HF, y el resto paresen ser trash values probablemente por singularidades en las matrices. Y comparando con las soluciones positivas del GEVP original, me parece que en este igualmente hay varios valores espureos mezcaldos con el que debiera ser el espectro real.
No recuerdo bien como se comparan los autovectores en este caso. No creo que hubiera la correspondencia que en HF con el factor de 1/2 como diferencia. Lo que si sucede en ambos casos es que la matriz de overlap sigue sin ser diagonal. Me parece que tengo que aplicar un procedimiento de ortogonalizacion intemedio o de normalizacion en el que solo mantenga los terminos del autovector que corresponden a configuraciones hh.
La normalizacion de la 2rdm reconstruida es tiene un factor de 1/2 respecto al resultado que se obtiene con GEVP original, y no se resultve el problema de que no es la normalizacion correcta. Con respecto a la componente bb de la traza de la 2rdm reconstruida en este caso, no se pierde totalmente como pasa con HF, pero es bien sparse, en tanto que la diagonal de la componente alphaalpha es aprox 1/2 de la diagonal de la que se obtiene con el procedimiento para el GEVP original.

A verificar:

Por que pierdo componente beta cuando recostruyo la 2rdm con este procedimiento de tomar solo estados p>q si en todo momento parece que tengo componentes aa, ab y ba en las matrices y vectores.

Ver que da este termino: tv[rs,pq] += rdm[i] * rdm[j], para cada estado independientemente antes de que se sume sobre todos ellos. Esto pudiera explicar por que cuando se usa DOCI referencia la traza de 2rdm recosntruida tiene submatriz alfa != de la beta para hh-ERPA. puede que al sumar los terminos tv[rs,pq] correspondientes a cada estado, haya cambios de signo que terminen causando esta diferencia entre los sectores aa y bb. Comparar con el caso en que HF es la referencia. -->

Check metric matrices of hh-ERPA with and without commutator on the RHS:

$P-matrix = <\Psi|p^\dagger q^\dagger s r|\Psi>$

$P-matrix -> Q$

$P-matrix -> G$

Test normalization conditions and symmetry

In [ ]:
from eomee.tools import two_positivity_condition, matrix_P_to_matrix

In [ ]:
hh2rpa = run_eom(nparts, h_ij, g_ijkl, one_dm=None, two_dm=None, operator='hh2', restricted=True, tol=1.0e-7, orthog="asymmetric")

In [ ]:
# hh2rhs = hh2rpa['gevp'].rhs
temp = hh2rpa['dm2']
answer = two_positivity_condition('p', k, hh2rpa['dm1'], hh2rpa['dm2'])
assert np.allclose(temp, answer)
print(np.einsum('abab', temp), np.einsum('abab', answer))

In [ ]:
# RHS: <|p+ q+ s r|> - <|s r p+ q+|>
hhrpa = run_eom(nparts, h_ij, g_ijkl, one_dm=None, two_dm=None, operator='hh', restricted=True, tol=1.0e-7, orthog="asymmetric")
hhrhs = hhrpa['gevp'].rhs

pmat = two_positivity_condition('p', k, hhrpa['dm1'], hhrpa['dm2'])
qmat = two_positivity_condition('q', k, hhrpa['dm1'], hhrpa['dm2'])
answer = pmat - qmat  #.transpose((3,2,1,0)) #srqp ->

assert np.allclose(hhrhs.reshape((n,n,n,n)), answer)

In [ ]:
# temp = ph2rhs.reshape((n,n,n,n))
# answer = two_positivity_condition('g', k, hh2rpa['dm1'], hh2rpa['dm2'])
# assert np.allclose(temp, answer)
# print(np.einsum('abab', temp), np.einsum('abab', answer), 4*(n-4+1))
# assert np.allclose(temp, temp.transpose((2,3,0,1)))
# assert np.allclose(answer, answer.transpose((2,3,0,1)))


# temp = pp2rhs.reshape((n,n,n,n))
# answer = two_positivity_condition('q', k, pp2rpa['dm1'], pp2rpa['dm2'])
# print(np.einsum('abab', temp), np.einsum('abab', answer), (n-4)*(n-4-1))
# assert np.allclose(temp, answer)


# # RHS: <|p+ q s+ r|> - <|s+ r p+ q|>
# phrpa = run_eom(nparts, h_ij, g_ijkl, one_dm=None, two_dm=None, operator='ph', restricted=True, tol=1.0e-7, orthog="asymmetric")
# phrhs = phrpa['gevp'].rhs
# interm = two_positivity_condition('g', k, ph2rpa['dm1'], ph2rpa['dm2'])
# answer = interm - interm.transpose((3,2,1,0))

# assert np.allclose(phrhs.reshape((n,n,n,n)), answer)


# # RHS: <|p q s+ r+|> - <|s+ r+ p q|>
# pprpa = run_eom(nparts, h_ij, g_ijkl, one_dm=None, two_dm=None, operator='pp', restricted=True, tol=1.0e-7, orthog="asymmetric")
# pprhs = pprpa['gevp'].rhs
# pmat = two_positivity_condition('p', k, pprpa['dm1'], pprpa['dm2'])
# qmat = two_positivity_condition('q', k, pprpa['dm1'], pprpa['dm2'])
# answer = qmat - pmat.transpose((3,2,1,0))

# assert np.allclose(pprhs.reshape((n,n,n,n)), answer)

In [ ]:
# # import matplotlib as mpl
# # from matplotlib import pyplot
# import matplotlib.pyplot as plt

# n, bins, patches = plt.hist(ev, density=False, bins=10, color ='blue')

# LHS symmetry problem

In [ ]:
from script_erpa import brute_hherpa_lhs

b_lhs = brute_hherpa_lhs(gevp['h'], gevp['v'], gevp['dm1'], gevp['dm2'])


In [ ]:
# alpha component
l_pqrs = b_lhs.reshape(n, n, n, n)
lhs_aa = l_pqrs[:k, :k, :k, :k]

# alpha beta component
lhs_ab = l_pqrs[:k, k:, :k, k:]

In [ ]:
assert np.allclose(l_pqrs, l_pqrs.transpose((2,3,0,1))) # kl ji, ji kl
assert np.allclose(l_pqrs, -l_pqrs.transpose((1,0,2,3))) # kl ji, lk ji
assert np.allclose(l_pqrs, -l_pqrs.transpose((0,1,3,2))) # kl ji, kl ij
assert np.allclose(l_pqrs, l_pqrs.transpose((1,0,3,2))) # kl ji, lk ij
assert np.allclose(l_pqrs, l_pqrs.transpose((3,2,1,0))) # kl ji, ji kl

In [ ]:
# Evaluate new 2-RDM and check trace
ev, cv = solve_dense(b_lhs, gevp['gevp'].rhs, tol=1.0e-7, orthog="asymmetric")

pes, pcv, _ = pickpositiveeig(ev, cv, tol=1.e-2)
print(np.sort(pes))

cv_mrs = pcv.reshape(pcv.shape[0], n, n)
tdms = np.einsum("mrs,pqsr->mpq", cv_mrs, gevp['gevp'].rhs.reshape(n,n,n,n)) #
tv = np.zeros_like(gevp['dm2']) #dm2
for rdm in tdms:
    tv += np.einsum("pq,rs->pqrs", rdm, rdm, optimize=True)
new_rdm2 =  tv/2

print('2-RDm normalization condition\n', np.einsum("ijij", new_rdm2), (sum(nparts) * (sum(nparts) - 1)))

# Observaciones recordatorio:

2 electrones, base minima (1/2 filling)

4 electrones base minima (1/2 filling)

sistemas 1/2 llenos

ref HF: rdm reconstruida normalizacion correcta, sin embargo en sistema de 4 particulas ya la componente alpha y beta de la traza de la nueva 2rdm no son son iguales (esto puede estar causado por el problema de no ortogonalidad de los estados) y al "corregir" la simetria de LHS las diferencias se hacen aum mayor. Tambien or alternativamente, es probable que este problema de diferencia entre componente alfa y beta de la traza de la nueva rdm pueda corregirse con el procedimiento en que solo se toman las configuraciones p>q, ya que al generar la nueva 2rdm se pondria `manualmente` los elementos de simetria esperados.

ref doci: realmente un desastre... yo esperaba que tener un modelo 1/2 lleno compensara las contribuciones de hh y pp existentes en el problema ERPA, pero no es el caso: para el modelo de 4 electrones la normalizacion de la nueva 2rdm es casi cero! cuando debiera dar 12. "corregir" la simetria de LHS nuevamete no resuelve ni la normalizacion, ni la diferencia entre las componentes alfa y beta de que he estado hablando


resumiendo elementos sospechosos de pp-ERPA:
- permutaciones de simetria incorrectas en LHS
- desigualdad de componentes alfa y beta de traza de 2rdm reconstruida a partir de TDMs de hh GEVP
- me cuestiono si hay algun efecto de la presencia de terminos no diagonales en la matriz metrica de hh-(e)RPA, aunq en el sistema de 2 electrones con base minima, no hay tales elementos y aun asi da mal para ERPA.